In [2]:
import sys
import os
import torch
import matplotlib.pyplot as plt
from datetime import datetime

# ✅ Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

# ✅ Change to your project directory
%cd /content/drive/MyDrive/RemoteSensing-Lora-InstructBlip

# # ✅ Add root directory to path so Python can find your modules
# sys.path.append(".")

# print(os.listdir("blip2_rsvqa_finetuning"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/RemoteSensing-Lora-InstructBlip


In [3]:
# Step 2: Install dependencies from requirements.txt
!pip install -r requirements.txt

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 106.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 43.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 13.5 MB/s eta 0:00:

In [4]:
# 1. 检查环境和GPU状态
import torch
import transformers
print(f"PyTorch version: {torch.__version__}")
print(f"Transformers version: {transformers.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")
    # 使用正确的内存查询API
    print(f"Total GPU Memory: {torch.cuda.get_device_properties(0).total_memory/1024**3:.2f}GB")
    print(f"Allocated GPU Memory: {torch.cuda.memory_allocated(0)/1024**3:.2f}GB")
    print(f"Reserved GPU Memory: {torch.cuda.memory_reserved(0)/1024**3:.2f}GB")

PyTorch version: 2.6.0+cu124
Transformers version: 4.53.2
CUDA available: True
GPU: NVIDIA A100-SXM4-40GB
Total GPU Memory: 39.56GB
Allocated GPU Memory: 0.00GB
Reserved GPU Memory: 0.00GB


In [5]:
# 2. 清理GPU内存
import gc
gc.collect()
torch.cuda.empty_cache()
if torch.cuda.is_available():
    print(f"Allocated Memory after cleanup: {torch.cuda.memory_allocated(0)/1024**3:.2f}GB")
    print(f"Reserved Memory after cleanup: {torch.cuda.memory_reserved(0)/1024**3:.2f}GB")

Allocated Memory after cleanup: 0.00GB
Reserved Memory after cleanup: 0.00GB


In [6]:
# 3. 尝试使用最小的InstructBLIP模型
from transformers import InstructBlipProcessor, InstructBlipForConditionalGeneration
from PIL import Image
import requests

# 使用flan-t5-xl版本（更小）
model_name = "Salesforce/instructblip-flan-t5-xl"
print(f"Loading model: {model_name}")

# 使用float16减少内存使用
model = InstructBlipForConditionalGeneration.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)
processor = InstructBlipProcessor.from_pretrained(model_name)

print("Model loaded successfully")

Loading model: Salesforce/instructblip-flan-t5-xl


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/833 [00:00<?, ?B/s]

Model loaded successfully


In [7]:
# 4. 将模型移到GPU并检查内存
device = "cuda" if torch.cuda.is_available() else "cpu"
model = model.to(device)
print(f"Model on {device}")
if device == "cuda":
    print(f" Memory after model loading: {torch.cuda.memory_reserved(0)/1024**3:.2f}GB")

Model on cuda
 Memory after model loading: 9.10GB


In [8]:
# 5. 测试简单的图像
# url = "https://raw.githubusercontent.com/salesforce/LAVIS/main/docs/_static/Confusing-Pictures.jpg"
image = Image.open("./RSGPT/dataset/RSIEval/images/P1384_0054.png").convert("RGB")
# image = Image.open(requests.get(url, stream=True).raw).convert("RGB")
prompt = "All the cars on the road?"

# 准备输入
print("Processing inputs...")
inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)
print("Inputs prepared")

Processing inputs...
Inputs prepared


In [4]:
import torch
# 6. 使用最简单的生成参数
print("Starting generation...")
import time
start_time = time.time()

# # 设置超时
# with torch.no_grad():
#     outputs = model.generate(
#         **inputs,
#         max_new_tokens=150,  # 使用max_new_tokens而不是max_length
#         do_sample=False,    # 关闭采样
#         num_beams=1         # 使用贪婪解码
#     )

# with torch.no_grad():
#     outputs = model.generate(
#       **inputs,
#       max_new_tokens=80,
#       do_sample=False,
#       num_beams=3,
#       repetition_penalty=1.2,
#       no_repeat_ngram_size=3,
#       early_stopping=True  # 遇到结束符就停止
#   )

# with torch.no_grad():
#      outputs = model.generate(
#       **inputs,
#       max_new_tokens=50,
#       do_sample=True,  # 开启采样
#       temperature=0.7,  # 控制随机性
#       top_p=0.9,  # nucleus采样
#       repetition_penalty=1.5
#   )

with torch.no_grad():
    outputs = model.generate(
      **inputs,
      max_new_tokens=300,  # RSGPT用这个
      num_beams=1,         # RSGPT对话用1
      do_sample=True,      # RSGPT对话用True
      top_p=0.9,
      repetition_penalty=1.0,  # RSGPT对话用1.0
      temperature=1.0
  )

end_time = time.time()
print(f"Generation completed in {end_time - start_time:.2f} seconds")

# 解码输出
generated_text = processor.batch_decode(outputs, skip_special_tokens=True)[0].strip()
print(f"\nGenerated text: {generated_text}")

Starting generation...


NameError: name 'model' is not defined